In [10]:
# paquetes de la implementación
import numpy as np
import ale_py
import gymnasium as gym
import random

# paquetes de las gráficas
import matplotlib.pyplot as plt
import cv2

In [ ]:
import gymnasium as gym
import cv2
import numpy as np
import random
import ale_py
import matplotlib.pyplot as plt

# -------------------------------
# 1. Creación del entorno y parámetros
# -------------------------------
env = gym.make('ALE/Breakout-v5')
num_acciones = env.action_space.n  # Número de acciones
# En Breakout, la observación es una imagen (por ejemplo, 210x160x3)
# Usaremos preprocesamiento para reducirla

# Parámetros de Dyna-Q
alpha = 0.1         # Tasa de aprendizaje
gamma = 0.995        # Factor de descuento
epsilon = 0.3       # Mayor exploración inicial (puedes ajustar)
n_planning = 50    # Pasos de planeación por paso real
num_episodios = 500

# Variables para llevar registro
pasos_totales = 0
lista_pasos = []         # Pasos acumulados al final de cada episodio
lista_recompensas = []   # Recompensa total por episodio

# -------------------------------
# 2. Preprocesamiento y discretización del estado
# -------------------------------
def preprocess_observation(obs):
    """
    Convierte la imagen de Breakout (RGB) a una imagen binaria de 16x16.
    """
    # Convertir a escala de grises (usando coeficientes estándar)
    gray = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
    # Redimensionar a 16x16 (interpolación por área)
    small = cv2.resize(gray, (16, 16), interpolation=cv2.INTER_AREA)
    # Binarizar: píxeles mayores que 128 se convierten en 1, el resto en 0
    binary = (small > 128).astype(np.uint8)
    return binary

def get_state_key(obs):
    """
    Procesa la observación y la convierte en una tupla, que servirá de clave.
    """
    processed = preprocess_observation(obs)
    return tuple(processed.flatten())

# -------------------------------
# 3. Inicialización de la tabla Q y el modelo
# -------------------------------
# Tabla Q: diccionario con clave: (estado, acción) y valor: Q(s,a)
Q = {}

def obtener_Q(estado, accion):
    key = (estado, accion)
    if key not in Q:
        Q[key] = 0.0
    return Q[key]

def actualizar_Q(estado, accion, td_error):
    key = (estado, accion)
    if key not in Q:
        Q[key] = 0.0
    Q[key] += alpha * td_error

# Modelo: diccionario que mapea (estado, acción) -> (recompensa, siguiente_estado)
Model = {}

# -------------------------------
# 4. Entrenamiento con Dyna-Q
# -------------------------------
for episodio in range(num_episodios):
    # env.reset() devuelve (obs, info)
    obs, info = env.reset()
    estado = get_state_key(obs)
    terminado = False
    recompensa_total = 0

    while not terminado:
        # Selección de acción con política ε-greedy
        if np.random.rand() < epsilon:
            accion = np.random.randint(num_acciones)
        else:
            q_vals = [obtener_Q(estado, a) for a in range(num_acciones)]
            accion = int(np.argmax(q_vals))
        
        # Ejecutar la acción en el entorno
        next_obs, recompensa, terminated, truncated, info = env.step(accion)
        terminal = terminated or truncated
        pasos_totales += 1
        recompensa_total += recompensa

        next_estado = get_state_key(next_obs)

        # Actualización de Q mediante Q-learning
        best_next_q = max([obtener_Q(next_estado, a) for a in range(num_acciones)])
        td_error = recompensa + gamma * best_next_q - obtener_Q(estado, accion)
        actualizar_Q(estado, accion, td_error)

        # Actualizar el modelo con la transición observada
        Model[(estado, accion)] = (recompensa, next_estado)

        # Planeación: simulamos transiciones del modelo
        if len(Model) > 0:
            for _ in range(n_planning):
                key_sim = random.choice(list(Model.keys()))
                r_sim, next_state_sim = Model[key_sim]
                a_sim = key_sim[1]
                best_next_q_sim = max([obtener_Q(next_state_sim, a) for a in range(num_acciones)])
                td_error_sim = r_sim + gamma * best_next_q_sim - obtener_Q(key_sim[0], a_sim)
                actualizar_Q(key_sim[0], a_sim, td_error_sim)
        
        estado = next_estado
        terminado = terminal

    lista_recompensas.append(recompensa_total)
    lista_pasos.append(pasos_totales)
    
    if (episodio + 1) % 100 == 0:
        print(f"Episodio {episodio+1}, Recompensa = {recompensa_total}")

env.close()

# -------------------------------
# 5. Gráfica: Recompensa Promedio vs. Pasos
# -------------------------------
def promedio_movil(datos, ventana=10):
    return np.convolve(datos, np.ones(ventana)/ventana, mode='valid')

ventana = 10
recompensas_suavizadas = promedio_movil(lista_recompensas, ventana)
pasos_suavizados = lista_pasos[ventana-1:]  # Alinear el promedio móvil

plt.figure(figsize=(8,5))
plt.plot(pasos_suavizados, recompensas_suavizadas, label=f"Recompensa Promedio (ventana={ventana})")
plt.xlabel("Pasos Acumulados")
plt.ylabel("Recompensa Promedio")
plt.title("Dyna-Q Tabular en Breakout (ALE)")
plt.legend()
plt.grid(True)
plt.show()

# -------------------------------
# 6. Evaluación de la Política Final
# -------------------------------
def evaluar_politica(env, Q, n_episodios=10):
    recompensas = []
    for _ in range(n_episodios):
        obs, info = env.reset()
        estado = get_state_key(obs)
        terminado = False
        recompensa_total = 0
        while not terminado:
            q_vals = [obtener_Q(estado, a) for a in range(num_acciones)]
            accion = int(np.argmax(q_vals))
            next_obs, recompensa, terminated, truncated, info = env.step(accion)
            terminal = terminated or truncated
            recompensa_total += recompensa
            estado = get_state_key(next_obs)
            terminado = terminal
        recompensas.append(recompensa_total)
    return recompensas

env_eval = gym.make('ALE/Breakout-v5')
recompensas_eval = evaluar_politica(env_eval, Q, n_episodios=10)
env_eval.close()

media_recompensa = np.mean(recompensas_eval)
std_recompensa = np.std(recompensas_eval)

print("\nResultados de la política final en 10 experimentos:")
print("----------------------------------------------")
print(f"| {'Métrica':<12} | {'Valor':<20} |")
print("----------------------------------------------")
print(f"| {'Media':<12} | {media_recompensa:<20.2f} |")
print(f"| {'Desviación':<12} | {std_recompensa:<20.2f} |")
print("----------------------------------------------")
